In [1]:
import torch
print(torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import sys
from answer_generators.wiki_answer_generator import WikiAnswerGenerator
from question_generators.simple_yes_question_generator import SimpleYesQuestionGenerator
from qa_utils import QuestionAnswerWriter
from question_generators.t5_question_generator import T5QuestionGenerator

True


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pabma\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Load dataset
from OLD_dataloader import *

# Load data using CustomDataOld function
data_path = 'H:/My Drive/2023-FALL/11-611 NLP/Project/Question_Answer_Dataset_v1.2/'
dataset = CustomDataOld(data_path)

length of dataset:  2725


In [3]:
#Generate set of articles
articles_set = set()

# Loop through the array and count each element
for i in range(len(dataset)):
    articles_set.add(dataset.article_name[i])
    
articles_set=list(articles_set)
articles_set.sort()
print(articles_set)

['Abraham_Lincoln', 'Alessandro_Volta', 'Amedeo_Avogadro', 'Anders_Celsius', 'Ant', 'Antwerp', 'Arabic_language', 'Bee', 'Beijing', 'Berlin', 'Blaise_Pascal', 'Butterfly', 'Calvin_Coolidge', 'Canada', 'Cello', 'Charles-Augustin_de_Coulomb', 'Chinese_language', 'Copenhagen', 'Cougar', 'Cymbal', 'Dhaka', 'Drum', 'Eel', 'Egypt', 'Finland', 'Finnish_language', 'Flute', 'Fox', 'Gerald_Ford', 'Ghana', 'Giant_Panda', 'Giraffe', 'Gray_Wolf', 'Grover_Cleveland', 'Guitar', 'Henri_Becquerel', 'Indonesia', 'Isaac_Newton', 'Jakarta', 'James_Monroe', 'James_Watt', 'John_Adams', 'Korean_language', 'Kuala_Lumpur', 'Liechtenstein', 'Lima', 'Lobster', 'London', 'Lyre', 'Malay_language', 'Melbourne', 'Michael_Faraday', 'Millard_Fillmore', 'Montreal', 'Nairobi', 'Nassau', 'Nikola_Tesla', 'Octopus', 'Ottawa', 'Otter', 'Piano', 'Portuguese_language', 'Qatar', 'Romania', 'Saint_Petersburg', 'San_Francisco', 'Santiago', 'Singapore', 'Swahili_language', 'Swan', 'Swedish_language', 'Taipei', 'Theodore_Roosevelt

In [4]:
#Write human generated questions and answers
path='files/'

article_name=articles_set[3] #couldn't read [2]

article=''

#get all human questions and answers for article
human_questions=[]
human_answers=[]
for i in range(len(dataset)):
    if article_name==dataset.article_name[i]:
        article=dataset[i][6]
        human_questions.append(dataset.questions[i])
        human_answers.append(dataset.answers[i])

print('Article',article_name)
print('Human Questions:',len(human_questions))
print('Human Answers:',len(human_answers))

#write human questions
with open(path+article_name+'_human_questions.txt', 'w') as file:
    for element in human_questions:
        file.write(element + "\n")

#write human answers
with open(path+article_name+'_human_answers.txt', 'w') as file:
    for element in human_answers:
        file.write(element + "\n")

#write article
with open(path+article_name+'.txt', 'w') as file:
    file.write(article + "\n")


Article Anders_Celsius
Human Questions: 16
Human Answers: 16


In [5]:
# Question output file
questions_filename=path+article_name+'_machine_questions.txt' 
qa_writer = QuestionAnswerWriter(question_filename=questions_filename)

In [6]:
# Machine Question Generation - downloads files from Wikipedia
# should change to reading from file in QA set for comparisons?
question_generator = T5QuestionGenerator(article_name)

successfully loaded checkpoint


In [7]:
#generate machine questions
machine_questions = question_generator.generate_questions()

context is: 
['Anders Celsius (Swedish pronunciation: [ˈânːdɛʂ ˈsɛ̌lːsɪɵs]; 27 November 1701 – 25 April 1744) was a Swedish astronomer, physicist and mathematician.', 'He was professor of astronomy at Uppsala University from 1730 to 1744, but traveled from 1732 to 1735 visiting notable observatories in Germany, Italy and France.', 'He founded the Uppsala Astronomical Observatory in 1741, and in 1742 proposed (an inverted form of) the Centigrade temperature scale which was later renamed Celsius in his honour.', 'Early life and education Anders Celsius was born in Uppsala, Sweden, on 27 November 1701.', 'His family originated from Ovanåker in the province of Hälsingland.', 'Their family estate was at Doma, also known as Höjen or Högen (locally as Högen 2).', "The name Celsius is a latinization of the estate's name (Latin celsus 'mound').", 'As the son of an astronomy professor, Nils Celsius, nephew of botanist Olof Celsius and the grandson of the mathematician Magnus Celsius and the astr

In [8]:
#Print generated machine questions
print(machine_questions)

#Writes machine generated questions to file
qa_writer.start_new_session(mode=QuestionAnswerWriter.QUESTION)
qa_writer.write_questions_to_file(machine_questions)

['When was Anders Celsius born?', 'When did Galileo visit Germany?', 'In what year was the Centigrade temperature scale proposed?', 'When was Anders Celsius born?', 'Where did his family originate from?', 'Where was the family estate located?', 'What does celsus mean in English?', "Who was Nils Celsius's uncle?", 'Who was a talented mathematician from an early age?', 'Where did Anders Celsius study?']


In [9]:
# Machine Answer generator model initialization from self generated questions
answer_generator = WikiAnswerGenerator(article_name)

Some weights of the model checkpoint at deepset/bert-base-cased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [43]:
#generate and write machine generated answers to questions in file

#qa_writer.start_new_session(mode=QuestionAnswerWriter.QUESTION_AND_ANSWER)
#for question in generated_questions:
#    answer = answer_generator.generate_answer(question)
#    qa_writer.write_question_answer_pair_to_file(question, answer)

Token indices sequence length is longer than the specified maximum sequence length for this model (643 > 512). Running this sequence through the model will result in indexing errors


All models failed to generate an answer.
All models failed to generate an answer.


In [10]:
#write answers for human questions

machine_answers=[]
for question in human_questions:
    print('Q:',question)
    answer = answer_generator.generate_answer(question)
    print('A:',answer)
    machine_answers.append(answer)


#write machine generated answers to human questions
with open(path+article_name+'_machine_answers.txt', 'w') as file:
    for element in machine_answers:
        file.write(element + "\n")
    

Q: When did Anders Celcius publish his observations on the aurora borealis?
A:  1733
Q: When did Anders Celcius publish his observations on the aurora borealis?
A:  1733
Q: What did Anders Celsius determine about the boiling of water?
A:  0
Q: What did Anders Celsius determine about the boiling of water?
A:  0
Q: Which temperature scale did Celsius propose?
A:  Centigrade
Q: Which temperature scale did Celsius propose?
A:  Centigrade
Q: What is the boiling point of water dependent on?
All models failed to generate an answer.
A: CLS
Q: What is the boiling point of water dependent on?
All models failed to generate an answer.
A: CLS
Q: Where was Celsius born?
A:  Uppsala, Sweden
Q: Where was Celsius born?
A:  Uppsala, Sweden
Q: Whad did Celsius report in his paper "Observations of two persistent degrees on a thermometer"?
All models failed to generate an answer.
A: CLS
Q: How old was Celsius when he died?
All models failed to generate an answer.
A: CLS
Q: How old was Celsius when he died?

# Question Analysis

In [11]:
# Question analysis

#elements
print(article_name)
print(human_questions)
print(machine_questions) #machine questions

Anders_Celsius
['When did Anders Celcius publish his observations on the aurora borealis?', 'When did Anders Celcius publish his observations on the aurora borealis?', 'What did Anders Celsius determine about the boiling of water?', 'What did Anders Celsius determine about the boiling of water?', 'Which temperature scale did Celsius propose?', 'Which temperature scale did Celsius propose?', 'What is the boiling point of water dependent on?', 'What is the boiling point of water dependent on?', 'Where was Celsius born?', 'Where was Celsius born?', 'Whad did Celsius report in his paper "Observations of two persistent degrees on a thermometer"?', 'How old was Celsius when he died?', 'How old was Celsius when he died?', 'What is temperature would water have to be to be halfway between its standard boiling and freezing point?', "What is the difference between Celsius' original temperature scale and the one we use today?", "What is the difference between Celsius' original temperature scale an

In [12]:
import importlib
import error_metrics  # Import the module itself, not the individual items
importlib.reload(error_metrics)
from error_metrics import *  # Re-import to update the namespace

In [13]:
## QUESTION ANALYSIS - Syntactic: contrast structure of human vs machine

#Human Question Analysis

with open(path+article_name+'_question_syntactic_analysis.txt', 'w') as file:

      header=('article_name',
            'human/machine',
            'question',
            'is_yes_no_type_question',
            'question_type',
            'word_count', 'n_content_words', 'n_relevant_words', 'conciseness_score',
            'has_verb', 'has_subject', 'has_reasonable_length', 'n_unique_POS','has_coherent_structure', 'fluency_score' 
            )

      file.write('|'.join(header) + "\n")

      for i in range(len(human_questions)):
            
            line=(article_name, #article name
                  'human', #human/machine,
                  human_questions[i], #question,
                  is_yes_no_question(human_questions[i]), #is yes/no type
                  classify_question(human_questions[i]), #question_type
                  *evaluate_conciseness(human_questions[i], detail=True), #word count, n content words, n relevant words, conciseness score
                  *evaluate_syntax_fluency(human_questions[i],detail=True) #'has_verb', 'has_subject', 'has_reasonable_length', 'has_unique_POS','has_coherent_structure', 'fluency_score'
                  )

            # Convert all elements to strings, converting True/False to 1/0, join them with a comma, and write to the file
            line = [str(int(item)) if isinstance(item, bool) else str(item) for item in line]
            file.write('|'.join(line) + "\n")

#Machine Question Analysis

with open(path+article_name+'_question_syntactic_analysis.txt', 'a') as file:

      for i in range(len(machine_questions)):
            
            line=(article_name, #article name
                  'machine', #human/machine,
                  machine_questions[i], #question,
                  is_yes_no_question(machine_questions[i]), #is yes/no type
                  classify_question(machine_questions[i]), #question_type
                  *evaluate_conciseness(machine_questions[i], detail=True), #word count, n content words, n relevant words, conciseness score
                  *evaluate_syntax_fluency(machine_questions[i],detail=True) #'has_verb', 'has_subject', 'has_reasonable_length', 'has_unique_POS','has_coherent_structure', 'fluency_score'
                  )

            # Convert all elements to strings, converting True/False to 1/0, join them with a comma, and write to the file
            line = [str(int(item)) if isinstance(item, bool) else str(item) for item in line]
            file.write('|'.join(line) + "\n")


In [14]:
## QUESTION ANALYSIS - Semantic: compare content

with open(path+article_name+'_question_semantic_analysis.txt', 'w') as file:
    
      header=('article_name',
            'machine_question',
            'human_question',
            'is_yes_no_machine_question',
            'type_machine_question',
            'is_yes_no_human_question',
            'type_human_question',
            'match_yes_no_type_question',
            'match_type_question',

            'f1_char_score',
            #'bleu_score',
            'jaccard_similarity',
            'jaccard_similarity_bigrams',
            'overlap_coefficient'
            )
      
      file.write('|'.join(header) + "\n")

      for machine in machine_questions:
            for human in human_questions:
                  line=(article_name, #article name
                        machine,      #machine questions
                        human,         #human question
                        is_yes_no_question(machine), #is yes/no type machine question
                        classify_question(machine), #type machine question
                        is_yes_no_question(human), #is yes/no human question
                        classify_question(human), #type human question,
                        is_yes_no_question(machine)==is_yes_no_question(human), #match yes/no type question
                        classify_question(machine)==classify_question(human), #match type question
                        
                        calculate_f1_character_score(human,machine),
                        #calculate_bleu_score(human, machine)
                        get_jaccard_similarity(human,machine),
                        get_jaccard_similarity_bigrams(human,machine),
                        get_overlap_coefficient(human,machine)
                        )
                              # Convert all elements to strings, converting True/False to 1/0, join them with a comma, and write to the file
                  line = [str(int(item)) if isinstance(item, bool) else str(item) for item in line]
                  file.write('|'.join(line) + "\n")


# Answer Analysis

In [16]:
## Answer Analysis - Semantic and Syntactic

with open(path+article_name+'_answer_consolidated_analysis.txt', 'w') as file:
      header=('article_name',
      'human_question',
      'is_yes_no_type_question',
      'question_type',
      'human_answer',
      'machine_answer',
      'is_yes_no_type_human_answer',
      'is_yes_no_type_machine_answer',
      'is_coherent_type_human_answer',
      'is_coherent_type_machine_answer',
      
      #Semantic
      'f1_char_score',
      #'bleu_score',
      'jaccard_similarity',
      'jaccard_similarity_bigrams',
      'overlap_coefficient',
      
      #Syntactic human
      'word_count_human', 'n_content_words_human', 'n_relevant_words_human', 'conciseness_score_human',
      'has_verb_human', 'has_subject_human', 'has_reasonable_length_human', 'n_unique_POS_human','has_coherent_structure_human', 'fluency_score_human', 
      
      #Syntactic machine
      'word_count_machine', 'n_content_words_machine', 'n_relevant_words_machine', 'conciseness_score_machine',
      'has_verb_machine', 'has_subject_machine', 'has_reasonable_length_machine', 'n_unique_POS_machine','has_coherent_structure_machine', 'fluency_score_machine' 
      
      )

      file.write('|'.join(header) + "\n")
      
      for i in range(len(human_questions)):
      
            line=(article_name, #article name
                  human_questions[i], #human question,
                  is_yes_no_question(human_questions[i]), #is yes/no type
                  classify_question(human_questions[i]), #question_type
                  human_answers[i], #human answer,
                  machine_answers[i], #machine answer,
                  is_yes_no_answer(human_answers[i]), #is yes/no type human
                  is_yes_no_answer(machine_answers[i]), #is yes/no type machine
                  is_yes_no_question(human_questions[i])==is_yes_no_answer(human_answers[i]), #'is_coherent_type_human_answer',
                  is_yes_no_question(human_questions[i])==is_yes_no_answer(machine_answers[i]), #'is_coherent_type_machine_answer',
                  
                  #Semantic
                  calculate_f1_character_score(human_answers[i],machine_answers[i]),
                  #calculate_bleu_score(human, machine)
                  get_jaccard_similarity(human_answers[i],machine_answers[i]),
                  get_jaccard_similarity_bigrams(human_answers[i],machine_answers[i]),
                  get_overlap_coefficient(human_answers[i],machine_answers[i]),

                  #Syntactic human
                  *evaluate_conciseness(human_answers[i], detail=True), #word c,ount, n content words, n relevant words, conciseness score
                  *evaluate_syntax_fluency(human_answers[i],detail=True), #'has_verb', 'has_subject', 'has_reasonable_length', 'has_unique_POS','has_coherent_structure', 'fluency_score'
            
                  #Syntactic machine
                  *evaluate_conciseness(machine_answers[i], detail=True), #word count, n content words, n relevant words, conciseness score
                  *evaluate_syntax_fluency(machine_answers[i],detail=True) #'has_verb', 'has_subject', 'has_reasonable_length', 'has_unique_POS','has_coherent_structure', 'fluency_score'
            
                  )

            # Convert all elements to strings, converting True/False to 1/0, join them with a comma, and write to the file
            line = [str(int(item)) if isinstance(item, bool) else str(item) for item in line]
            file.write('|'.join(line) + "\n")                                       



TypeError: Value after * must be an iterable, not int

In [4]:
!pip install pandas matplotlib seaborn


DEPRECATION: Loading egg at c:\users\pabma\anaconda3\lib\site-packages\torchdata-0.7.1-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at c:\users\pabma\anaconda3\lib\site-packages\torchtext-0.16.1-py3.11-win-amd64.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set style for seaborn
sns.set_theme()

# Read the CSV files into DataFrames
df_q_syntactic = pd.read_csv('files/Abraham_Lincoln_question_syntactic_analysis.txt',sep='|')   
df_q_semantic = pd.read_csv('files/Abraham_Lincoln_question_semantic_analysis.txt',sep='|')    
df_a_general = pd.read_csv('files/Abraham_Lincoln_answer_consolidated_analysis.txt',sep='|')      






In [18]:
df_q_syntactic.groupby(['question_type','human/machine']).count()

article_name  question  is_yes_no_type_question  \
question_type  human/machine                                                    
How            human                     4         4                        4   
Open-Ended     human                     2         2                        2   
               machine                   1         1                        1   
What/Where/Who human                     9         9                        9   
               machine                   8         8                        8   
When/Why       human                     9         9                        9   
               machine                   1         1                        1   
Yes/No         human                    21        21                       21   

                              word_count  n_content_words  n_relevant_words  \
question_type  human/machine                                                  
How            human                   4                4                 4   
Open-Ended     human                   2                2                 2   
               machine                 1                1                 1   
What/Where/Who human                   9                9                 9   
               machine                 8                8                 8   
When/Why       human                   9                9                 9   
               machine                 1                1                 1   
Yes/No         human                  21               21                21   

                              conciseness_score  has_verb  has_subject  \
question_type  human/machine                                             
How            human                          4         4            4   
Open-Ended     human                          2         2            2   
               machine                        1         1            1   
What/Where/Who human                          9         9            9   
               machine                        8         8            8   
When/Why       human                          9         9            9   
               machine                        1         1            1   
Yes/No         human                         21        21           21   

                              has_reasonable_length  n_unique_POS  \
question_type  human/machine                                        
How            human                              4             4   
Open-Ended     human                              2             2   
               machine                            1             1   
What/Where/Who human                              9             9   
               machine                            8             8   
When/Why       human                              9             9   
               machine                            1             1   
Yes/No         human                             21            21   

                              has_coherent_structure  fluency_score  
question_type  human/machine                                         
How            human                               4              4  
Open-Ended     human                               2              2  
               machine                             1              1  
What/Where/Who human                               9              9  
               machine                             8              8  
When/Why       human                               9              9  
               machine                             1              1  
Yes/No         human                              21             21

In [16]:
percentage_df = (df_q_syntactic.groupby(['question_type', 'human/machine'])
                             .size()
                             .groupby(level=0)
                             .apply(lambda x: 100 * x / x.sum())
                             .reset_index(name='percentage'))

# Plotting
plt.figure(figsize=(12, 6))
sns.barplot(x='question_type', y='percentage', hue='human/machine', data=percentage_df)
plt.title('Percentage of Question Types by Human/Machine')
plt.xlabel('Question Type')
plt.ylabel('Percentage of Total Cases (%)')
plt.xticks(rotation=45)  # Rotates the labels on the x-axis for better readability
plt.show()

ValueError: cannot insert question_type, already exists